In [139]:
import os
from sklearn import svm
from mlens.ensemble import SuperLearner
from mlens.metrics.metrics import rmse
from sklearn.linear_model import Lasso
from sklearn.svm import *
import numpy as np
import pandas as pd
from scipy.stats import uniform, randint
from sklearn.metrics import accuracy_score,f1_score
import math
from sklearn.model_selection import *
from mlens.model_selection import Evaluator
import sklearn.ensemble
import random
from sklearn import metrics
import sys
from sklearn.model_selection import GridSearchCV
import easy_excel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from mlens.metrics import make_scorer
from mlens.preprocessing import EnsembleTransformer

In [158]:

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
def svm_crossvalidation():
    datapath ='/home02/chenhuangrong/KNN.csv'
    train_data = pd.read_csv(datapath, header=None, index_col=None)
    X = np.array(train_data)
    Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
    Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
    Y.extend(Y2)
    Y = np.array(Y)
    print X.shape
    print len(Y)
#     x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1,random_state=33)
    meta_learners = [SVC(), ('rf', RandomForestClassifier(random_state=2))]
    base_learners = [RandomForestClassifier(random_state=random.randint(1,100)),
             SVC(probability=True)]

    proba_transformer = EnsembleTransformer().add('blend', base_learners, proba=True)
    class_transformer = EnsembleTransformer().add('blend', base_learners, proba=False)
    f1_scorer = make_scorer(f1_score, average='micro', greater_is_better=True)
    preprocessing = {'proba': [('layer-1', proba_transformer)],
             'class': [('layer-1', class_transformer)]}
    # Set parameter mapping
    # Here, we differentiate distributions between cases for the random forest
    params = {'svc': {'C': uniform(0, 10)},
              ('class', 'rf'): {'max_depth': randint(2, 3)},
              ('proba', 'rf'): {'max_depth': randint(2, 3),
                                'max_features': uniform(0.5, 0.5)}
              }
    evaluator = Evaluator(scorer=f1_scorer, random_state=2017, cv=10,n_jobs=1,shuffle=True)
    if __name__=='__main__':
        evaluator.fit(X, Y, meta_learners, params, preprocessing=preprocessing, n_iter=2)
#         sum_rmse=0
#         X_positive=X[0:len(X)/2]
#         X_negative=X[len(X)/2:]
#         kf = KFold(n_splits=10,shuffle=True)  # positive and negative samples will be shuffled 
#         for train_index , test_index in kf.split(X_positive):
#                 X_positive=pd.DataFrame(X_positive)
#                 X_negative=pd.DataFrame(X_negative)
#                 x_train_positive=X_positive.iloc[train_index,:]
#                 x_test_positive=X_positive.iloc[test_index,:]
#                 x_train_negative=X_negative.iloc[train_index,:]
#                 x_test_negative=X_negative.iloc[test_index,:]
#                 x_train=pd.concat([x_train_positive,x_train_negative],axis=0)
#                 x_test=pd.concat([x_test_positive,x_test_negative],axis=0)
#                 y_train_positive=[1 for e in xrange(len(x_train)/2)]
#                 y_train_negative=[0 for e in xrange(len(x_train)/2)]
#                 y_test_positive=[1 for e in xrange(len(x_test)/2)]
#                 y_test_negative=[0 for e in xrange(len(x_test)/2)]
#                 y_train=y_train_positive
#                 y_train.extend(y_train_negative)
#                 y_test=y_test_positive
#                 y_test.extend(y_test_negative)  
#                 ensemble=SuperLearner()
#                 preprossing_cases={'mm':[MinMaxScaler()],'sc':[StandardScaler()]}
#                 estimators_per_case={'mm':[SVR()],'sc':[('can name some or all ests',Lasso())]}
#                 ensemble.add(estimators_per_case,preprossing_cases).add_meta(SVC())
#                 ensemble.fit(x_train,y_train)
#                 y_pred=ensemble.predict(x_test)
#                 sum_rmse=accuracy_score(y_pred=y_pred,y_true=y_test)+sum_rmse
#         print sum_rmse/10
if __name__ == '__main__':
    svm_crossvalidation()


(2614, 20)
2614


ValueError: The number of classes has to be greater than one; got 1